# Assignment 2: Evolutionary dynamics in a spatial context

## Snowdrift

In [ ]:
import random
import plotly.graph_objs as go
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode() 
import copy
import numpy as np

In [ ]:
HEIGHT = 50
WIDTH = 50

In [ ]:
T=10
R=7
P=0
S=3

### Matrix creation

In [ ]:
def printm(matrix):
    for y in range(len(matrix)):
        for x in range(len(matrix[0])):
            print(matrix[x][y], end=" ")
        print("")

In [ ]:
def create_random_matrix():
    result = []
    for i in range(WIDTH):
        row = []
        for j in range(HEIGHT):
            if random.randrange(2): # 0 ou 1
                row.append("D")
            else:
                row.append("C")
        result.append(row)
    return result

In [ ]:
def create_payoff_matrix():
    return [[0 for j in range(HEIGHT)] for i in range(WIDTH)]

In [ ]:
def count_c(matrix):
    n = 0
    for x in range(len(matrix)):
        for y in range(len(matrix[0])):
            if matrix[x][y] == "C":
                n += 1
    return n

### Neighbors

In [ ]:
def get_von_neuman_neighbors(x, y):
    neighbors = []
    neighbors.append([x - 1, y])
    neighbors.append([x, y - 1])
    neighbors.append([(x + 1)%WIDTH, y])
    neighbors.append([x, (y + 1)%HEIGHT])
    return neighbors


In [ ]:
def get_moore_neighbors(x, y): 
    neighbors = []
    neighbors.append([x - 1, y])
    neighbors.append([x, y - 1])
    neighbors.append([(x + 1)%WIDTH, y])
    neighbors.append([x, (y + 1)%HEIGHT])
    neighbors.append([x - 1, y-1])
    neighbors.append([(x + 1)%WIDTH, y-1])
    neighbors.append([x - 1, (y + 1)%HEIGHT])
    neighbors.append([(x + 1)%WIDTH, (y + 1)%HEIGHT])
    return neighbors

### Payoff

In [ ]:
def payoff_p1_vs_p2(choice1, choice2):
    """Payoff for player 1 for a single game"""
    if (choice1 == "C"):
        if (choice2 == "C"):
            return R
        else:
            return S
    else:
        if (choice2 == "C"):
            return T
        else:
            return P

In [ ]:
def payoff_for_p(matrix, p , nf = get_moore_neighbors):
    """Payoff of a  case in the matrix"""
    neighbors = nf(p[0], p[1])
    payoff = 0
    for n in neighbors:
        payoff += payoff_p1_vs_p2(matrix[p[0]][p[1]], matrix[n[0]][n[1]])
    return payoff

In [ ]:
def payoff_for_all(matrix):
    payoffm = create_payoff_matrix()
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            p = (i,j)
            payoffm[i][j] = payoff_for_p(matrix, p)
    return payoffm

### Imitation

In [ ]:
def new_strategy_for_p(matrix, p , payoffmat, nf = get_moore_neighbors):
    neighbor = random.choice(nf(p[0], p[1]))
    payoff_p = payoffmat[p[0]][p[1]]
    strategy_p = matrix[p[0]][p[1]]
    payoff_n = payoffmat[neighbor[0]][neighbor[1]]
    strategy_n = matrix[neighbor[0]][neighbor[1]]
    p = ( 1 + (payoff_n-payoff_p) / (4*(max([P,R,T,S])-min([P,R,T,S]))) )/ 2
    if p <= random.uniform(0,1):
        return strategy_n
    return strategy_p

In [ ]:
def new_strategy_for_all(matrix, payoffm):
    newmatrix = copy.deepcopy(matrix)
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            p = (i,j)
            newmatrix[i][j] = new_strategy_for_p(matrix, p, payoffm)
    return newmatrix

### Simulation

In [ ]:
def simulation(n=100):
    matrix = create_random_matrix()
    res = []
    res.append(count_c(matrix) / (HEIGHT * WIDTH))
    for i in range(n):
        payoffm = payoff_for_all(matrix)
        matrix = new_strategy_for_all(matrix, payoffm)
        res.append(count_c(matrix) / (HEIGHT * WIDTH))
    return res

In [ ]:
def get_one_matrix(n=50):
    matrix = create_random_matrix()
    res = []
    res.append(count_c(matrix) / (HEIGHT * WIDTH))
    for i in range(n):
        payoffm = payoff_for_all(matrix)
        matrix = new_strategy_for_all(matrix, payoffm)
        res.append(count_c(matrix) / (HEIGHT * WIDTH))
    return matrix

### Exo

In [ ]:
r = np.average([simulation() for i in range(100)], axis=0)

In [ ]:
# Create a trace
trace = go.Scatter(
    x = [i for i in range(len(r))],
    y = r
)

data = [trace]

# Plot and embed in ipython notebook!
plotly.offline.iplot(data)

In [ ]:
matrix = get_one_matrix()

In [ ]:
tracehm = go.Heatmap(
    z= [[0 if i=="D" else 1 for i in j] for j in matrix],
    x=[i+1 for i in range(len(matrix))],
    y=[i+1 for i in range(len(matrix[0]))]
)
data = [tracehm]
plotly.offline.iplot(data)